In [1]:
import taichi as ti
import torch
from eincasm_python.eincasm import Eincasm

VIS_CHIDS = [('com', 'r'), ('com', 'g'), ('com', 'b')]

@ti.dataclass
class VisParams:
    chindices: ti.types.vector(n=len(VIS_CHIDS), dtype=ti.i32)
    chlims: ti.types.matrix(n=len(VIS_CHIDS), m=2, dtype=ti.f32)
    scale: ti.i32
    brush_radius: ti.i32
    chnum_to_paint: ti.i32
    chindex_to_paint: ti.i32
    val_to_paint: ti.f32
    val_to_paint_dt: ti.f32
    drawing: ti.i32
    mouse_posx: ti.f32
    mouse_posy: ti.f32
    perturb_strength: ti.f32
    is_perturbing_weights: ti.i32
    is_perturbing_biases: ti.i32
    test: ti.f32

    def __init__(self,
                 chindices: ti.types.vector(n=len(VIS_CHIDS), dtype=ti.i32),
                 chlims: ti.types.matrix(n=len(VIS_CHIDS), m=2, dtype=ti.f32),
                 scale: ti.i32,
                 chindex_to_paint = None,
                 brush_radius = 4,
                 chnum_to_paint = 0,
                 val_to_paint = 1.5,
                 val_to_paint_dt = -1,
                 drawing = False,
                 mouse_posx = 0.0,
                 mouse_posy = 0.0,
                 perturb_strength = 0.1,
                 is_perturbing_weights = False,
                 is_perturbing_biases = False,
                 test = -1):
        if chindex_to_paint is None:
            chindex_to_paint = self.chindices[0]
        self.chindices = chindices
        self.chlims = chlims
        self.scale = scale
        self.chindex_to_paint = chindex_to_paint
        self.brush_radius = brush_radius
        self.chnum_to_paint = chnum_to_paint
        self.val_to_paint = val_to_paint
        self.val_to_paint_dt = val_to_paint_dt
        self.drawing = drawing
        self.mouse_posx = mouse_posx
        self.mouse_posy = mouse_posy
        self.perturb_strength = perturb_strength
        self.is_perturbing_weights = is_perturbing_weights
        self.is_perturbing_biases = is_perturbing_biases
        self.test = test

ein = Eincasm(shape=(100, 100), torch_device=torch.device("mps"), num_com=5)
world = ein.world
chindices = world.get_inds_tivec(VIS_CHIDS)
chlims = world.get_lims_timat(VIS_CHIDS)
VisParams(chindices=chindices, chlims=chlims, scale=1)

[Taichi] version 1.7.0, llvm 15.0.7, commit 6b563a2a, osx, python 3.10.8


[I 11/17/23 12:27:34.878 683263] [shell.py:_shell_pop_print@23] Graphical python shell detected, using wrapped sys.stdout


TaichiRuntimeError: Cannont create field, maybe you forgot to call `ti.init()` first?

In [2]:
ein = eincasm(shape=(400,400), torch_device=torch.device('mps'))
chids = [('com', 'r'), ('com', 'g'), ('com', 'b')]

/Users/aidanbx/CS/EINCASM/src_ti/eincasm.py:36: UserWarning: World: Casting portmap of dtype: torch.int8 to world dtype: torch.float32
  self.world.malloc()


In [19]:
inds = ein.world.get_inds_tivec(chids)
itype = ti.types.vector(inds.n, dtype=ti.i32)
istruct = ti.types.struct(lims=itype)
ifield = istruct.field(shape=())
ifield[None]['lims'] = inds

[W 11/14/23 20:12:11.017 12378070] 
Assign may lose precision: unknown <- i32
[W 11/14/23 20:12:11.021 12378070] 
Assign may lose precision: unknown <- i32
[W 11/14/23 20:12:11.023 12378070] 
Assign may lose precision: unknown <- i32


In [28]:
@ti.dataclass
class tstc:
    a: ti.i32
    b: ti.f32
    def init(self):
        self.a = 10
        self.b = 21

tfield = tstc.field(shape=())
tfield[None] = tstc()
tfield[None].init()
tfield[None].b

[W 11/14/23 22:04:37.802 12378070] 
Assign may lose precision: unknown <- i32
[W 11/14/23 22:04:37.814 12378070] 
Assign may lose precision: unknown <- f32


21.0

In [21]:
lims = ein.world.get_lims_timat(chids)
ltype = ti.types.matrix(lims.n, lims.m, dtype=ti.f32)
lstruct = ti.types.struct(lims=ltype)
lfield = lstruct.field(shape=())
lfield[None]['lims'] = lims

ValueError: Invalid data type <class 'taichi.lang.matrix.Matrix'>

In [12]:
ti.types.struct(
    flow=ti.types.vector(n=5, dtype=ti.f32),
    port=ti.f32,
    mine=ti.f32,)